<a href="https://colab.research.google.com/github/aabitokh/home_repo/blob/develop/ntb_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [75]:
#!pip install transformers
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement catboost-gpu (from versions: none)
ERROR: No matching distribution found for catboost-gpu


In [61]:
import pandas as pd 
import numpy as np 

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

import torch
import transformers
from tqdm import notebook
from torch import tensor


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re

from catboost import Pool, CatBoostClassifier

In [3]:
DATA_PATH = 'toxic_comments.csv'
#DATA_PATH = '/datasets/toxic_comments.csv'

In [11]:
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z\' ]', ' ', text).split())

In [12]:
#проверка на GPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
!nvidia-smi

Fri Mar 17 14:09:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# first look 

In [34]:
try:
    df = pd.read_csv(DATA_PATH)
except pd.errors.ParserError:
    df = pd.read_csv(DATA_PATH, encoding='utf-8', engine='python', delimiter=',' )


In [37]:
#df=df.sample(1292)

In [38]:
df.sample(10)

,Unnamed: 0,text,toxic
70578,70648,The Spiderman 4 Teaser Poster \n\nI'm taking i...,0
45010,45063,THATS what i was talking about. It's a fact th...,0
138192,138343,"Ok, ta. I'll just add it to the Discussion pag...",0
125836,125967,History of Organized Jewish Wikipedians \n\nHe...,0
17520,17537,"""\n\n why are you trying to make me really mad...",0
150335,150491,I have added a FU rationale to the image. Jus...,0
84025,84104,You keep arguing that you are right and keep m...,0
123706,123834,""" \nErrant, it's at a delicate stage and I'm w...",0
104931,105028,Deleting the article would very improper on yo...,0
125464,125593,"Here. I don't agree with it, but could I/we at...",0


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


# data preprocessing

In [40]:
df = df.drop(['Unnamed: 0'], axis = 1) 

In [41]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [42]:
df.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [43]:
df['text'] = df['text'].apply(lambda x: clear_text(x))

# BERT

In [44]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

#model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
tokenized = df['text'].apply(lambda x: \
                             tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500))

In [46]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [47]:
padded = torch.LongTensor(padded).to(DEVICE)
attention_mask = torch.LongTensor(attention_mask).to(DEVICE)
model.to(DEVICE)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [48]:
!nvidia-smi

Fri Mar 17 14:20:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    35W /  70W |   5367MiB / 15360MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [49]:
padded.shape

torch.Size([159292, 500])

In [50]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range((padded.shape[0] // batch_size)+1)):
    batch = padded[batch_size*i:batch_size*(i+1)]
    attention_mask_batch = attention_mask[batch_size*i:batch_size*(i+1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/1593 [00:00<?, ?it/s]

In [53]:
features = np.concatenate(embeddings)
features

array([[ 0.17851065, -0.0731292 , -0.0242072 , ...,  0.09886129,
         0.3328736 ,  0.36018914],
       [-0.04720632, -0.13178937,  0.34290305, ..., -0.10028522,
         0.53204   ,  0.34827146],
       [ 0.11508776,  0.124497  , -0.09554707, ..., -0.00473316,
         0.6559014 ,  0.28874958],
       ...,
       [-0.0775509 , -0.07158832, -0.04552566, ..., -0.1766507 ,
         0.64981055,  0.28371865],
       [ 0.14775652, -0.07487001,  0.06121582, ...,  0.01370696,
         0.3257963 ,  0.28902218],
       [ 0.20483875,  0.03516744, -0.16541754, ..., -0.08565068,
         0.40237337,  0.27500334]], dtype=float32)

# classification

In [54]:
x_train, x_test, y_train, y_test = train_test_split(features, df['toxic'], test_size= 0.2)

## LR 

In [59]:
lr = LogisticRegression()
scores = cross_val_score(lr, x_train, y_train, cv = 3, scoring= 'f1')

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

0.7347264032914747


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [62]:
print(scores.mean())

0.7347264032914747


## ctb

In [101]:
%%time 

X_train_ctb = x_train.copy()

y_train_ctb = y_train.copy()

params = {'iterations': 500, 'depth':10,  'verbose':False,
          'class_weights': [1, 10], 'task_type': 'GPU'}

params = {'iterations': 500,
          'learning_rate': 0.1,
          'depth': 4,
          'l2_leaf_reg': 3,
          'random_seed': 42,
          'eval_metric': 'F1',
          'task_type': 'GPU', 'verbose':False}

model = CatBoostClassifier(**params)

scores = cross_val_score(model, X_train_ctb, y_train_ctb, 
                         cv = 3, scoring= 'f1')

CPU times: user 1min 5s, sys: 11.8 s, total: 1min 16s
Wall time: 48.4 s


In [102]:
scores.mean()

0.6519578196892932

In [99]:
model.fit(X_train_ctb, y_train_ctb, )
pred = model.predict(x_test)

f1_score(pred, y_test)

0.6999649491763057